# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298455167710                   -0.85    5.4
  2   -8.300191060636       -2.76       -1.25    1.0
  3   -8.300449694633       -3.59       -1.89    3.4
  4   -8.300462967964       -4.88       -2.78    1.8
  5   -8.300464563345       -5.80       -3.08    3.6
  6   -8.300464599434       -7.44       -3.24    7.6
  7   -8.300464621171       -7.66       -3.39    1.1
  8   -8.300464633216       -7.92       -3.53    1.0
  9   -8.300464640685       -8.13       -3.71    1.0
 10   -8.300464641700       -8.99       -3.78    2.2
 11   -8.300464643280       -8.80       -3.98    1.1
 12   -8.300464644008       -9.14       -4.80    1.1


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67374237854                   -0.70    6.0
  2   -16.67790147866       -2.38       -1.14    1.2
  3   -16.67918937364       -2.89       -1.87    3.6
  4   -16.67924583978       -4.25       -2.65    4.1
  5   -16.67928568879       -4.40       -3.11    4.4
  6   -16.67928620489       -6.29       -3.51    1.8
  7   -16.67928621898       -7.85       -4.06    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32399800338                   -0.56    7.0
  2   -33.33245352378       -2.07       -1.00    1.1
  3   -33.33586134348       -2.47       -1.71    5.4
  4   -33.33623606626       -3.43       -2.58    7.2
  5   -33.33684383122       -3.22       -2.33    7.8
  6   -33.33685340867       -5.02       -2.36    1.0
  7   -33.33694343643       -4.05       -3.49    3.2
  8   -33.33694346531       -7.54       -3.49    3.5
  9   -33.33694389224       -6.37       -4.07    2.2


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298400286346                   -0.85    5.0
  2   -8.300248525671       -2.73       -1.59    1.0
  3   -8.300442686265       -3.71       -2.50    5.0
  4   -8.300346562429   +   -4.02       -2.22   11.0
  5   -8.300464550470       -3.93       -3.76    2.9
  6   -8.300464596964       -7.33       -3.96    2.6
  7   -8.300464636533       -7.40       -4.33    1.8


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.31861937651                   -0.56    6.8
  2   -33.31679235260   +   -2.74       -1.27    1.0
  3   -21.02692139766   +    1.09       -0.52    7.9
  4   -33.29636001815        1.09       -1.69    7.0
  5   -33.13591115551   +   -0.79       -1.30    5.0
  6   -32.53486333430   +   -0.22       -1.11    4.6
  7   -33.31985452392       -0.11       -1.84    4.5
  8   -33.33460222927       -1.83       -2.16    2.2
  9   -33.33547305591       -3.06       -2.19    2.9
 10   -33.33664994566       -2.93       -2.71    2.0
 11   -33.33693173108       -3.55       -3.22    3.4
 12   -33.33693671951       -5.30       -3.44    3.9
 13   -33.33693872084       -5.70       -3.60    3.0
 14   -33.33694309642       -5.36       -3.95    2.0
 15   -33.33693182726   +   -4.95       -3.52    4.0
 16   -33.33694388122       -4.92       -4.54    4.4


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.